# 第八章C：Pandas教程
____

## 学习目标
1. 学习`Series`和`DataFrame`这两种常用的数据结构的初始化和按行、按列操作。

2. 掌握针对`DataFrame`的一些数据库操作:`concatenate`、`merge`、`join`、`groupby`和`aggregate`等

3. 对一些数据文件实例进行分析。

首先，明确Pandas是数据科学中非常有用的针对常见数据表格的操作的工具。

In [1]:
import numpy as np
import pandas as pd

## 1. 常用数据结构

### 1.1 Series

- Series是一种类似List的数据结构，但可以有名字，也就是所谓的index

In [2]:
salaries = pd.Series([12000, 15000, 13000, 13500], index=["Beijing", "Hongkong", "Shanghai", "Shenzhen"], name="salary")
print(salaries)
print()
print(type(salaries))

Beijing     12000
Hongkong    15000
Shanghai    13000
Shenzhen    13500
Name: salary, dtype: int64

<class 'pandas.core.series.Series'>


In [3]:
?pd.Series

- `pd.Series`这个初始化函数的参数主要包括：
  * data
  * index
  * dtype
  * name

- Series可以用类似list的方法进行切片（单个元素返回的是np.int等类型；多个元素返回的还是Series）

In [4]:
salaries[1], type(salaries[1])

(15000, numpy.int64)

In [5]:
salaries[1:], type(salaries[1:])

(Hongkong    15000
 Shanghai    13000
 Shenzhen    13500
 Name: salary, dtype: int64,
 pandas.core.series.Series)

In [6]:
salaries["Beijing":"Shanghai"]

Beijing     12000
Hongkong    15000
Shanghai    13000
Name: salary, dtype: int64

In [7]:
salaries.index

Index(['Beijing', 'Hongkong', 'Shanghai', 'Shenzhen'], dtype='object')

In [8]:
gdp = pd.Series([35000, 47000, 42000, 38000], index=salaries.index, name="gdp")
population = pd.Series([2400, 2000, 2600, 1800], index=salaries.index, name="pop")

### 1.2 DataFrame

`DataFrame`对应的是表的数据结构。

- 我们可将多个具有相同index的Series合并为一个DataFrame：

In [9]:
cities = pd.DataFrame([salaries,gdp,population])

In [10]:
type(cities)

pandas.core.frame.DataFrame

In [11]:
pd.DataFrame?

In [12]:
print(cities)

        Beijing  Hongkong  Shanghai  Shenzhen
salary    12000     15000     13000     13500
gdp       35000     47000     42000     38000
pop        2400      2000      2600      1800


- 对于一个DataFrame来说，有两个重要的属性：`columns`和`index`，我们可以理解为列索引和行索引

In [13]:
cities.columns

Index(['Beijing', 'Hongkong', 'Shanghai', 'Shenzhen'], dtype='object')

In [14]:
cities.index

Index(['salary', 'gdp', 'pop'], dtype='object')

- 还可以对DataFrame进行转置操作：

In [15]:
cities= cities.T

In [16]:
print(cities)

          salary    gdp   pop
Beijing    12000  35000  2400
Hongkong   15000  47000  2000
Shanghai   13000  42000  2600
Shenzhen   13500  38000  1800


In [17]:
cities.index

Index(['Beijing', 'Hongkong', 'Shanghai', 'Shenzhen'], dtype='object')

- 我们也可以从dict开始，构建我们的DataFrame：

In [18]:
cities = pd.DataFrame({
    "salaries":[12000, 15000, 13000, 13500],
    "gdp":[35000, 47000, 42000, 38000],
    "population":[2400, 2000, 2600, 1800]
}, index=["Beijing", "Hongkong", "Shanghai", "Shenzhen"])

In [19]:
print(cities)

          salaries    gdp  population
Beijing      12000  35000        2400
Hongkong     15000  47000        2000
Shanghai     13000  42000        2600
Shenzhen     13500  38000        1800


- DataFrame可以针对列和行进行索引：
  * 用`<df>.iloc[<rows>]`使用数值对行和列进行索引，这里的`i`是`integer`的意义。
  * 用`<df>.loc[<rows>, <cols>]`使用索引值对行和列进行索引，但不能用数值。
  * 这里的`<rows>`和`<cols>`索引可以是范围、列表、条件。

In [20]:
cities["salaries"], type(cities["salaries"])

(Beijing     12000
 Hongkong    15000
 Shanghai    13000
 Shenzhen    13500
 Name: salaries, dtype: int64,
 pandas.core.series.Series)

- `iloc[]`采用数值范围进行索引

In [21]:
cities.iloc[1:3]

,salaries,gdp,population
Hongkong,15000,47000,2000
Shanghai,13000,42000,2600


- `iloc[]`也可用布尔数组进行索引：

In [22]:
cities.iloc[list(cities.salaries >= 13000)]

,salaries,gdp,population
Hongkong,15000,47000,2000
Shanghai,13000,42000,2600
Shenzhen,13500,38000,1800


- 想要了解更多相关信息，请参考帮助

In [23]:
cities.iloc?

- 相比而言，`loc`不使用数值，而是使用索引值对行列进行索引（如果只想要行，列用`:`；对列亦然）

- 用`<start>:<end>`确定范围，`<start>`和`<end>`都可以为空。

In [24]:
cities.loc[:, "salaries":"gdp"]

,salaries,gdp
Beijing,12000,35000
Hongkong,15000,47000
Shanghai,13000,42000
Shenzhen,13500,38000


- 可以用列表

In [25]:
cities.loc["Beijing":"Shanghai", ["salaries","population"]]

,salaries,population
Beijing,12000,2400
Hongkong,15000,2000
Shanghai,13000,2600


- 也可以用条件

In [26]:
cities.loc[cities.salaries >= 13000, :]

,salaries,gdp,population
Hongkong,15000,47000,2000
Shanghai,13000,42000,2600
Shenzhen,13500,38000,1800


In [27]:
cities.at["Shanghai", "gdp"]

42000

In [28]:
cities.iat[2, 2]

2600

## 2. 索引index

- index也是一种对象
- index的值是不能被修改的，是immutable的object

In [29]:
cities.index, type(cities.index)

(Index(['Beijing', 'Hongkong', 'Shanghai', 'Shenzhen'], dtype='object'),
 pandas.core.indexes.base.Index)

In [30]:
cities.columns, type(cities.columns)

(Index(['salaries', 'gdp', 'population'], dtype='object'),
 pandas.core.indexes.base.Index)

In [31]:
cities.index[3] = "Guangzhou"

TypeError: Index does not support mutable operations

- 如果要修改，只能整体修改（也就是创建一个新的Index对象，而不能对原有的Index对象进行修修补补的工作）

In [32]:
cities.index = ["Beijing", "Hongkong", "Shanghai", "Guangzhou"]

### 2.1 可以对Index进行索引和切片操作

In [33]:
obj = pd.Series(np.arange(4), index=["a", "b", "c", "d"])
print(obj)

a    0
b    1
c    2
d    3
dtype: int64


In [34]:
obj[["d", "a"]]

d    3
a    0
dtype: int64

In [36]:
obj[[3,0]]

d    3
a    0
dtype: int64

In [37]:
obj[1:3] = 8

In [38]:
print(obj)

a    0
b    8
c    8
d    3
dtype: int64


In [39]:
obj["b":"d"] = 5
print(obj)

a    0
b    5
c    5
d    5
dtype: int64


- 对于DataFrame的索引和切片操作具有相似的手段

In [40]:
frame = pd.DataFrame(np.random.rand(3,3), 
                    index=["a", "b", "c"],
                    columns=["x", "y", "z"])
print(frame)

          x         y         z
a  0.738499  0.343550  0.299903
b  0.118977  0.620358  0.636288
c  0.511598  0.105139  0.491480


In [41]:
frame["a":"b"]

,x,y,z
a,0.738499,0.343550,0.299903
b,0.118977,0.620358,0.636288


In [42]:
frame.loc[:,"x":"y"]

,x,y
a,0.738499,0.343550
b,0.118977,0.620358
c,0.511598,0.105139


In [43]:
frame.iloc[:-2, :-1]

,x,y
a,0.738499,0.34355


In [44]:
frame.iloc[1:3,1:2]

,y
b,0.620358
c,0.105139


### 2.2 reindex

> 一个Series或者DataFrame可以按照新的索引重新进行indexing

In [45]:
obj = pd.Series([4.5, 7.2, -5.3, 3.2], index=['d', 'b', 'a', 'c'])
print(obj)

d    4.5
b    7.2
a   -5.3
c    3.2
dtype: float64


In [46]:
obj.reindex(["a", "b", "c", "d", "e"])

a   -5.3
b    7.2
c    3.2
d    4.5
e    NaN
dtype: float64

- 重新建立索引的过程，可以对新的索引对应的空值进行填补：

In [47]:
obj.reindex(["a", "b", "c", "d", "e"], fill_value=0)

a   -5.3
b    7.2
c    3.2
d    4.5
e    0.0
dtype: float64

- 还有其他的填补方法，但只能针对数值索引

In [48]:
obj = pd.Series(['blue', 'purple', 'yellow'], index = [0,2,4])
print(obj)

0      blue
2    purple
4    yellow
dtype: object


In [49]:
obj.reindex(range(6), method="ffill")

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [50]:
obj.reindex(range(6), method="bfill").dropna()

0      blue
1    purple
2    purple
3    yellow
4    yellow
dtype: object

### 2.3 drop操作

- `drop`还可以对Series和DataFrame进行操作

In [51]:
print(cities)

           salaries    gdp  population
Beijing       12000  35000        2400
Hongkong      15000  47000        2000
Shanghai      13000  42000        2600
Guangzhou     13500  38000        1800


In [53]:
cities.drop(["Beijing","Guangzhou"], axis=0)

,salaries,gdp,population
Hongkong,15000,47000,2000
Shanghai,13000,42000,2600


In [54]:
cities.drop("population", axis=1)

,salaries,gdp
Beijing,12000,35000
Hongkong,15000,47000
Shanghai,13000,42000
Guangzhou,13500,38000


### 2.4 多层索引Hierarchical indexing
- Series和DataFrame都允许存在多层索引
- 这是一个Series多层索引的例子：

In [55]:
data = pd.Series(np.random.randn(10), 
                 index=[['a','a','a','b','b','c','c','c','d','d'], \
                        [1,2,3,1,2,1,2,3,1,2]])
print(data)

a  1   -3.219242
   2   -1.183128
   3    1.025086
b  1   -0.008995
   2   -1.619569
c  1   -0.222849
   2    2.225497
   3    0.579028
d  1   -0.390985
   2    0.461026
dtype: float64


In [56]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('c', 1),
            ('c', 2),
            ('c', 3),
            ('d', 1),
            ('d', 2)],
           )

In [57]:
print(data["a"])

1   -3.219242
2   -1.183128
3    1.025086
dtype: float64


In [58]:
print(data[1:4])

a  2   -1.183128
   3    1.025086
b  1   -0.008995
dtype: float64


- `unstack`可以将`Hierarchical indexing`转换为DataFrame：

In [59]:
dfdata = data.unstack()
print(dfdata)

          1         2         3
a -3.219242 -1.183128  1.025086
b -0.008995 -1.619569       NaN
c -0.222849  2.225497  0.579028
d -0.390985  0.461026       NaN


- `stack()`可以将DataFrame转换为Hierarchical indexing：

In [60]:
dfdata.stack()

a  1   -3.219242
   2   -1.183128
   3    1.025086
b  1   -0.008995
   2   -1.619569
c  1   -0.222849
   2    2.225497
   3    0.579028
d  1   -0.390985
   2    0.461026
dtype: float64

- 这种数据在统计上称为`stacked data`

## 3. 数据库操作

这里我们将介绍几种常用的与数据库相关的操作：
- Concatenation
- Merge
- Join
- Aggregate

### 3.1 `pandas.concat()`

In [61]:
df1 = pd.DataFrame({'housing': [55000, 60000],
                   'cars': [200000, 300000],},
                  index = ['Shanghai', 'Beijing'])
df1

,housing,cars
Shanghai,55000,200000
Beijing,60000,300000


In [62]:
df2 = pd.DataFrame({'housing': [25000, 20000],
                   'cars': [150000, 120000],},
                  index = ['Hangzhou', 'Najing'])
print(df2)

          housing    cars
Hangzhou    25000  150000
Najing      20000  120000


In [63]:
df3 = pd.DataFrame({'housing': [30000, 10000],
                   'cars': [180000, 100000],},
                  index = ['Guangzhou', 'Chongqing'])
print(df3)

           housing    cars
Guangzhou    30000  180000
Chongqing    10000  100000


In [64]:
df = [df1, df2, df3]
morecities = pd.concat(df)
print(morecities)

           housing    cars
Shanghai     55000  200000
Beijing      60000  300000
Hangzhou     25000  150000
Najing       20000  120000
Guangzhou    30000  180000
Chongqing    10000  100000


- 在进行连接的时候可以加入`keys`参数，这样可以为不同的部分加入新的index，这样就形成了hierarchical indexing。

In [65]:
morecities2 = pd.concat(df, keys=["first", "second", "third"])
print(morecities2)

                  housing    cars
first  Shanghai     55000  200000
       Beijing      60000  300000
second Hangzhou     25000  150000
       Najing       20000  120000
third  Guangzhou    30000  180000
       Chongqing    10000  100000


In [66]:
morecities2.iloc[3:6]

housing    cars
second Najing       20000  120000
third  Guangzhou    30000  180000
       Chongqing    10000  100000

- 除了按行连接，也可以按列连接：

In [67]:
df4 = pd.DataFrame({'salaries': [10000, 30000, 30000, 20000, 15000]},
                  index = ['Suzhou', 'Beijing', 'Shanghai', 'Guangzhou', 'Tianjin'])
print(df4)

           salaries
Suzhou        10000
Beijing       30000
Shanghai      30000
Guangzhou     20000
Tianjin       15000


In [ ]:
print(morecities)

In [68]:
morecities3 = pd.concat([morecities, df4], axis=1)
morecities3

,housing,cars,salaries
Shanghai,55000.0,200000.0,30000.0
Beijing,60000.0,300000.0,30000.0
Hangzhou,25000.0,150000.0,NaN
Najing,20000.0,120000.0,NaN
Guangzhou,30000.0,180000.0,20000.0
Chongqing,10000.0,100000.0,NaN
Suzhou,NaN,NaN,10000.0
Tianjin,NaN,NaN,15000.0


- 这里可以调用`stack()`方法将DataFrame转换为Hierarchical indexing的stacked set，避免了存在NaN：

In [69]:
morecities3.stack()

Shanghai   housing      55000.0
           cars        200000.0
           salaries     30000.0
Beijing    housing      60000.0
           cars        300000.0
           salaries     30000.0
Hangzhou   housing      25000.0
           cars        150000.0
Najing     housing      20000.0
           cars        120000.0
Guangzhou  housing      30000.0
           cars        180000.0
           salaries     20000.0
Chongqing  housing      10000.0
           cars        100000.0
Suzhou     salaries     10000.0
Tianjin    salaries     15000.0
dtype: float64

- 可以实现“Inner join”：

In [70]:
pd.concat([morecities, df4], axis=1, join="inner")

,housing,cars,salaries
Shanghai,55000,200000,30000
Beijing,60000,300000,30000
Guangzhou,30000,180000,20000


In [71]:
pd.concat([morecities, df4], axis=1)

,housing,cars,salaries
Shanghai,55000.0,200000.0,30000.0
Beijing,60000.0,300000.0,30000.0
Hangzhou,25000.0,150000.0,NaN
Najing,20000.0,120000.0,NaN
Guangzhou,30000.0,180000.0,20000.0
Chongqing,10000.0,100000.0,NaN
Suzhou,NaN,NaN,10000.0
Tianjin,NaN,NaN,15000.0


- 通过`inner join`，排除了包含`NaN`的记录；

- 有时候，我们还可以通过`append()`方法，实现行上面的`concatenate()`：

In [72]:
df1.append(df2)

,housing,cars
Shanghai,55000,200000
Beijing,60000,300000
Hangzhou,25000,150000
Najing,20000,120000


In [75]:
df1

,housing,cars
Shanghai,55000,200000
Beijing,60000,300000


In [73]:
df1.append(df4)

,housing,cars,salaries
Shanghai,55000.0,200000.0,NaN
Beijing,60000.0,300000.0,NaN
Suzhou,NaN,NaN,10000.0
Beijing,NaN,NaN,30000.0
Shanghai,NaN,NaN,30000.0
Guangzhou,NaN,NaN,20000.0
Tianjin,NaN,NaN,15000.0


- 但是很明显，这种`append`只是单纯实现行的叠加，而不能根据index来实现连接。

- DataFrame还可以与Series进行连接（Series先转化为DataFrame，再进行连接）：

In [76]:
s1 = pd.Series([20, 30], index=["Beijing", "Shanghai"], name="meal")
pd.concat([df1,s1], axis=1)

,housing,cars,meal
Shanghai,55000,200000,30
Beijing,60000,300000,20


### 3.2 `pandas.merge()`和`<df>.join()`

`merge()`可以实现根据某共有的列对两个DataFrame进行合并：

In [77]:
df1 = pd.DataFrame({'housing': [55000, 60000, 58000],
                   'cars': [200000, 300000,250000],
                  'cities': ['Shanghai', 'Beijing','Shenzhen']})
print(df1)

   housing    cars    cities
0    55000  200000  Shanghai
1    60000  300000   Beijing
2    58000  250000  Shenzhen


In [78]:
df2 = pd.DataFrame({'salaries': [10000, 30000, 30000, 20000, 15000],
                  'cities': ['Suzhou', 'Beijing', 'Shanghai', 'Guangzhou', 'Tianjin']})
print(df2)

   salaries     cities
0     10000     Suzhou
1     30000    Beijing
2     30000   Shanghai
3     20000  Guangzhou
4     15000    Tianjin


In [79]:
pd.merge(df1, df2, on="cities")         # inner join

,housing,cars,cities,salaries
0,55000,200000,Shanghai,30000
1,60000,300000,Beijing,30000


- 默认采用的是`inner join`，我们还可以更换为`outer join`或者`left join`和`right join`：

In [80]:
pd.merge(df1, df2, on="cities", how="outer") # outer join

,housing,cars,cities,salaries
0,55000.0,200000.0,Shanghai,30000.0
1,60000.0,300000.0,Beijing,30000.0
2,58000.0,250000.0,Shenzhen,NaN
3,NaN,NaN,Suzhou,10000.0
4,NaN,NaN,Guangzhou,20000.0
5,NaN,NaN,Tianjin,15000.0


In [81]:
pd.merge(df1, df2, on="cities", how="left") # left join

,housing,cars,cities,salaries
0,55000,200000,Shanghai,30000.0
1,60000,300000,Beijing,30000.0
2,58000,250000,Shenzhen,NaN


In [82]:
pd.merge(df1, df2, on="cities", how="right") # right join

,housing,cars,cities,salaries
0,55000.0,200000.0,Shanghai,30000
1,60000.0,300000.0,Beijing,30000
2,NaN,NaN,Suzhou,10000
3,NaN,NaN,Guangzhou,20000
4,NaN,NaN,Tianjin,15000


- 还可以根据索引进行join

In [83]:
df1 = pd.DataFrame({'housing': [55000, 60000, 58000],
                   'cars': [200000, 300000,250000]},
                  index=['Shanghai', 'Beijing','Shenzhen'])
print(df1)

          housing    cars
Shanghai    55000  200000
Beijing     60000  300000
Shenzhen    58000  250000


In [84]:
df2 = pd.DataFrame({'salaries': [10000, 30000, 30000, 20000, 15000]},
                  index=['Suzhou', 'Beijing', 'Shanghai', 'Guangzhou', 'Tianjin'])
print(df2)

           salaries
Suzhou        10000
Beijing       30000
Shanghai      30000
Guangzhou     20000
Tianjin       15000


In [85]:
df1.join(df2, how="outer")

,housing,cars,salaries
Beijing,60000.0,300000.0,30000.0
Guangzhou,NaN,NaN,20000.0
Shanghai,55000.0,200000.0,30000.0
Shenzhen,58000.0,250000.0,NaN
Suzhou,NaN,NaN,10000.0
Tianjin,NaN,NaN,15000.0


In [86]:
df1.join(df2, how="inner")

,housing,cars,salaries
Shanghai,55000,200000,30000
Beijing,60000,300000,30000


In [87]:
df1.join(df2, how="left")

,housing,cars,salaries
Shanghai,55000,200000,30000.0
Beijing,60000,300000,30000.0
Shenzhen,58000,250000,NaN


In [88]:
df1.join(df2, how="right")

,housing,cars,salaries
Suzhou,NaN,NaN,10000
Beijing,60000.0,300000.0,30000
Shanghai,55000.0,200000.0,30000
Guangzhou,NaN,NaN,20000
Tianjin,NaN,NaN,15000


- 这种情况的也同样可用`pd.merge()`实现：

In [89]:
pd.merge(df1, df2, left_index=True, right_index=True, how='inner')

,housing,cars,salaries
Shanghai,55000,200000,30000
Beijing,60000,300000,30000


### 3.3 `GroupBy()`和`Aggregate()`

我们用一个例子来说明GroupBy的用法。假设我们有一个部门几个人近年来的收入清单：

In [90]:
salaries = pd.DataFrame({
    'Name': ['Wang', 'Chen', 'Chen', 'Lin', 'Wang', 'Wang', 'Chen', 'Wang'],
    'Year': [2016,2016,2016,2016,2017,2017,2017,2017],
    'Salary': [10000,2000,4000,5000,18000,25000,3000,4000],
    'Bonus': [3000,1000,1000,1200,4000,2300,500,1000]
})
print(salaries)

   Name  Year  Salary  Bonus
0  Wang  2016   10000   3000
1  Chen  2016    2000   1000
2  Chen  2016    4000   1000
3   Lin  2016    5000   1200
4  Wang  2017   18000   4000
5  Wang  2017   25000   2300
6  Chen  2017    3000    500
7  Wang  2017    4000   1000


In [91]:
group_by_name = salaries.groupby('Name')
group_by_name

可以通过`aggregate`一块计算每个人的收入总额：

In [92]:
group_by_name.aggregate(sum)

,Year,Salary,Bonus
Name,,,
Chen,6049,9000,2500
Lin,2016,5000,1200
Wang,8067,57000,10300


当然，我们这里将年份也进行了加和，这显然是有问题的，该怎么调整呢？

我们还可以用`sum()`方法实现：

In [93]:
group_by_name.sum()

,Year,Salary,Bonus
Name,,,
Chen,6049,9000,2500
Lin,2016,5000,1200
Wang,8067,57000,10300


In [94]:
group_by_name_year = salaries.groupby(["Name", "Year"])
group_by_name_year.sum()

Salary  Bonus
Name Year               
Chen 2016    6000   2000
     2017    3000    500
Lin  2016    5000   1200
Wang 2016   10000   3000
     2017   47000   7300

In [95]:
group_by_name_year.max()   # min, size

Salary  Bonus
Name Year               
Chen 2016    4000   1000
     2017    3000    500
Lin  2016    5000   1200
Wang 2016   10000   3000
     2017   25000   4000

- 还可以用`describe()`展示其他统计信息：

In [96]:
group_by_name_year.describe()

Salary                                                         \
           count          mean           std      min      25%      50%   
Name Year                                                                 
Chen 2016    2.0   3000.000000   1414.213562   2000.0   2500.0   3000.0   
     2017    1.0   3000.000000           NaN   3000.0   3000.0   3000.0   
Lin  2016    1.0   5000.000000           NaN   5000.0   5000.0   5000.0   
Wang 2016    1.0  10000.000000           NaN  10000.0  10000.0  10000.0   
     2017    3.0  15666.666667  10692.676622   4000.0  11000.0  18000.0   

                            Bonus                                           \
               75%      max count         mean         std     min     25%   
Name Year                                                                    
Chen 2016   3500.0   4000.0   2.0  1000.000000     0.00000  1000.0  1000.0   
     2017   3000.0   3000.0   1.0   500.000000         NaN   500.0   500.0   
Lin  2016   5000.0   5000.0   1.0  1200.000000         NaN  1200.0  1200.0   
Wang 2016  10000.0  10000.0   1.0  3000.000000         NaN  3000.0  3000.0   
     2017  21500.0  25000.0   3.0  2433.333333  1504.43788  1000.0  1650.0   

                                   
              50%     75%     max  
Name Year                          
Chen 2016  1000.0  1000.0  1000.0  
     2017   500.0   500.0   500.0  
Lin  2016  1200.0  1200.0  1200.0  
Wang 2016  3000.0  3000.0  3000.0  
     2017  2300.0  3150.0  4000.0

## 4. 实例数据分析

我们这儿有一个`bike.csv`的文件，统计的是蒙特利尔7条自行车骑行路线每天骑行的人数记录。

In [97]:
bikes = pd.read_csv('bikes.csv', encoding='latin1', sep=';', 
                    parse_dates=['Date'], dayfirst=True, index_col='Date')

In [98]:
bikes.head()

,Berri 1,Brébeuf (données non disponibles),Côte-Sainte-Catherine,Maisonneuve 1,Maisonneuve 2,du Parc,Pierre-Dupuy,Rachel1,St-Urbain (données non disponibles)
Date,,,,,,,,,
2012-01-01,35,NaN,0,38,51,26,10,16,NaN
2012-01-02,83,NaN,1,68,153,53,6,43,NaN
2012-01-03,135,NaN,2,104,248,89,3,58,NaN
2012-01-04,144,NaN,1,116,318,111,8,61,NaN
2012-01-05,197,NaN,2,124,330,97,13,95,NaN


In [99]:
bikes.shape

(310, 9)

我们可以看到包含大量的NaN，考虑是不是应该把这些列给删除呢？

In [101]:
bikes.dropna(how='all', axis=1).head()

,Berri 1,Côte-Sainte-Catherine,Maisonneuve 1,Maisonneuve 2,du Parc,Pierre-Dupuy,Rachel1
Date,,,,,,,
2012-01-01,35,0,38,51,26,10,16
2012-01-02,83,1,68,153,53,6,43
2012-01-03,135,2,104,248,89,3,58
2012-01-04,144,1,116,318,111,8,61
2012-01-05,197,2,124,330,97,13,95


In [102]:
bikes_berri = bikes[["Berri 1"]]

In [104]:
bikes_berri_group_weekday = bikes_berri.groupby(bikes_berri.index.weekday)

In [105]:
bikes_berri_group_weekday.sum()

,Berri 1
Date,
0,134298
1,135305
2,152972
3,160131
4,141771
5,101578
6,99310


统计了该自行车骑行道周日-周六的总骑行人数。

In [106]:
bikes_berri_group_weekday.describe()

Berri 1                                                                  
       count         mean          std    min     25%     50%      75%     max
Date                                                                          
0       45.0  2984.400000  2093.291975   83.0  802.00  3292.0  4931.00  6206.0
1       44.0  3075.113636  2151.083580  135.0  897.25  3020.5  5165.00  6712.0
2       44.0  3476.636364  2302.985737  138.0  461.25  3734.0  5669.50  6529.0
3       44.0  3639.340909  2370.078754   92.0  839.50  4080.5  5757.00  6908.0
4       44.0  3222.068182  2288.254361   75.0  973.50  3681.5  5532.75  7077.0
5       44.0  2308.590909  1866.266405   32.0  644.00  1876.5  4130.25  5421.0
6       45.0  2206.888889  1771.607016   35.0  520.00  1940.0  3698.00  5255.0

这是我们针对一条车道的统计，如果我们要统计所有的车道呢？

首先我们计算每天骑车出行人数的总和：

In [107]:
bikes = bikes.dropna(axis=1, how="all")

In [108]:
bikes_sum = bikes.sum(axis=1).to_frame()
bikes_sum.columns = ["num_bikes"]
bikes_sum.head()

,num_bikes
Date,
2012-01-01,176
2012-01-02,407
2012-01-03,639
2012-01-04,759
2012-01-05,858


In [109]:
bikes_sum.loc[:, "weekday"] = bikes_sum.index.weekday

In [110]:
bikes_sum.head()

,num_bikes,weekday
Date,,
2012-01-01,176,6
2012-01-02,407,0
2012-01-03,639,1
2012-01-04,759,2
2012-01-05,858,3


In [111]:
bikes_sum_group_weekday = bikes_sum.groupby("weekday")
weekday_counts = bikes_sum_group_weekday.sum()
weekday_counts.index = ["Sunday", "Monday", "Tuesday", "Wednesday", 
                       "Thirsday", "Friday", "Saturday"]
weekday_counts

,num_bikes
Sunday,714963
Monday,698582
Tuesday,789722
Wednesday,829069
Thirsday,738772
Friday,516701
Saturday,518047


你看看其他你能做的么？

## 总结

对于Pandas模块，我们学习了
- 如何构造Series和DataFrame对象
- 如何访问Series和DataFrame中的子集
- 如何对Index进行操作
- 如何对Pandas数据集进行类SQL操作

到目前为止，我们已经能对数据集读取的基础上进行基本的处理和分析。